In [1]:
# To enable plotting graphs in Jupyter notebook
%matplotlib inline 
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split


In [2]:
Bank_df=pd.read_excel("Bank_Personal_Loan_Modelling.xlsx",sheetname='Data')

In [3]:
temp_df= Bank_df
Bank_df['Mortgage_Binary'] = Bank_df['Mortgage'].map(lambda x: 1 if x > 0 else 0)
Bank_df['Mortgage_Binary'].value_counts()
Bank_df=Bank_df.drop(['ID', 'ZIP Code','Mortgage'], axis=1)
Bank_df['Personal Loan']=Bank_df['Personal Loan'].astype('category')
Bank_df['Education']=Bank_df['Education'].astype('category')
Bank_df['Securities Account']=Bank_df['Securities Account'].astype('category')
Bank_df['CD Account']=Bank_df['CD Account'].astype('category')
Bank_df['Online']=Bank_df['Online'].astype('category')
Bank_df['CreditCard']=Bank_df['CreditCard'].astype('category')
Bank_df['Mortgage_Binary']=Bank_df['Mortgage_Binary'].astype('category')
Bank_df.dtypes

Age                      int64
Experience               int64
Income                   int64
Family                   int64
CCAvg                  float64
Education             category
Personal Loan         category
Securities Account    category
CD Account            category
Online                category
CreditCard            category
Mortgage_Binary       category
dtype: object

In [6]:
X = Bank_df[['Income','Family','CCAvg','Education','Securities Account','CD Account','Online','Mortgage_Binary']]
Y = Bank_df['Personal Loan']

In [7]:

test_size = 0.30 # taking 70:30 training and test set
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [11]:
dt_model = DecisionTreeClassifier(criterion = 'entropy', class_weight={0:.5,1:.5}, max_depth = 5, min_samples_leaf=5 )


In [12]:
lrcl = LogisticRegression(random_state=1)
rfcl = RandomForestClassifier(random_state=1)
nbcl = GaussianNB()
bgcl = BaggingClassifier(base_estimator=dt_model, n_estimators=90)  #the base_estimator can be null. The bagging classifer  will build it's own tree

enclf = VotingClassifier(estimators = [('lor', lrcl), ('rf', rfcl), ('nb', nbcl), ('bg', bgcl)], voting = 'hard')

In [13]:
for clf, label in zip([lrcl , rfcl, nbcl, enclf, bgcl], ['Logistic Regression', 'RandomForest', 'NaiveBayes', 'Ensemble', 'Bagging']):
    scores = cross_val_score(clf, X, Y, cv=5, scoring='accuracy')
    print("Accuracy: %0.02f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label ))

Accuracy: 0.95 (+/- 0.00) [Logistic Regression]
Accuracy: 0.98 (+/- 0.00) [RandomForest]
Accuracy: 0.89 (+/- 0.01) [NaiveBayes]
Accuracy: 0.96 (+/- 0.00) [Ensemble]
Accuracy: 0.99 (+/- 0.00) [Bagging]
